In [1]:
import os
import torch
import cv2
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import yaml
from pathlib import Path
import shutil
from ultralytics import YOLO
import albumentations as A
from albumentations import BboxParams
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURATION ===
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

NUM_CLASSES = 6
CONF_THRESHOLD = 0.25
IOU_THRESHOLD = 0.45
IMG_SIZE = 640

# Dataset paths
TRAIN_IMG_ROOT = "./train_images"
TRAIN_ANN_ROOT = "./train_annotations"
VAL_IMG_ROOT = "./test"
SUBMISSION_PATH = "./submission.csv"

# YOLO dataset structure
YOLO_DATASET_ROOT = "./yolo_dataset"
YOLO_TRAIN_IMG = os.path.join(YOLO_DATASET_ROOT, "images", "train")
YOLO_VAL_IMG = os.path.join(YOLO_DATASET_ROOT, "images", "val")
YOLO_TRAIN_LABEL = os.path.join(YOLO_DATASET_ROOT, "labels", "train")
YOLO_VAL_LABEL = os.path.join(YOLO_DATASET_ROOT, "labels", "val")

# === LABELS ===
LABEL_MAP = {
    'crazing': 0,
    'inclusion': 1,
    'patches': 2,
    'pitted_surface': 3,
    'rolled-in_scale': 4,
    'scratches': 5
}
REV_LABEL_MAP = {v: k for k, v in LABEL_MAP.items()}

class DatasetConverter:
    """Convert XML annotations to YOLO format"""
    
    def __init__(self, img_dir, ann_dir):
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        
    def xml_to_yolo_bbox(self, bbox, img_width, img_height):
        """Convert XML bbox to YOLO format (normalized center coordinates and dimensions)"""
        xmin, ymin, xmax, ymax = bbox
        
        # Calculate center coordinates and dimensions
        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        width = xmax - xmin
        height = ymax - ymin
        
        # Normalize coordinates
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height
        
        return [x_center, y_center, width, height]
    
    def convert_annotation(self, xml_path, img_path):
        """Convert single XML annotation to YOLO format"""
        # Get image dimensions
        img = Image.open(img_path)
        img_width, img_height = img.size
        
        # Parse XML
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        yolo_annotations = []
        for obj in root.findall('object'):
            label = obj.find('name').text
            if label not in LABEL_MAP:
                continue
                
            class_id = LABEL_MAP[label]
            
            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text)
            ymin = float(bbox.find('ymin').text)
            xmax = float(bbox.find('xmax').text)
            ymax = float(bbox.find('ymax').text)
            
            # Convert to YOLO format
            yolo_bbox = self.xml_to_yolo_bbox([xmin, ymin, xmax, ymax], img_width, img_height)
            yolo_annotations.append([class_id] + yolo_bbox)
            
        return yolo_annotations
    
    def create_yolo_dataset(self, train_split=0.8):
        """Create YOLO dataset structure"""
        # Create directories
        os.makedirs(YOLO_TRAIN_IMG, exist_ok=True)
        os.makedirs(YOLO_VAL_IMG, exist_ok=True)
        os.makedirs(YOLO_TRAIN_LABEL, exist_ok=True)
        os.makedirs(YOLO_VAL_LABEL, exist_ok=True)
        
        # Get all image files
        img_files = [f for f in os.listdir(self.img_dir) if f.endswith('.jpg')]
        img_files.sort()
        
        # Split dataset
        num_train = int(len(img_files) * train_split)
        train_files = img_files[:num_train]
        val_files = img_files[num_train:]
        
        print(f"Converting dataset: {len(train_files)} train, {len(val_files)} val")
        
        # Process training set
        for img_file in tqdm(train_files, desc="Converting train set"):
            img_path = os.path.join(self.img_dir, img_file)
            xml_path = os.path.join(self.ann_dir, img_file.replace('.jpg', '.xml'))
            
            if not os.path.exists(xml_path):
                continue
                
            # Copy image
            shutil.copy2(img_path, os.path.join(YOLO_TRAIN_IMG, img_file))
            
            # Convert and save annotation
            yolo_annotations = self.convert_annotation(xml_path, img_path)
            
            txt_file = img_file.replace('.jpg', '.txt')
            txt_path = os.path.join(YOLO_TRAIN_LABEL, txt_file)
            
            with open(txt_path, 'w') as f:
                for annotation in yolo_annotations:
                    f.write(' '.join(map(str, annotation)) + '\n')
        
        # Process validation set
        for img_file in tqdm(val_files, desc="Converting val set"):
            img_path = os.path.join(self.img_dir, img_file)
            xml_path = os.path.join(self.ann_dir, img_file.replace('.jpg', '.xml'))
            
            if not os.path.exists(xml_path):
                continue
                
            # Copy image
            shutil.copy2(img_path, os.path.join(YOLO_VAL_IMG, img_file))
            
            # Convert and save annotation
            yolo_annotations = self.convert_annotation(xml_path, img_path)
            
            txt_file = img_file.replace('.jpg', '.txt')
            txt_path = os.path.join(YOLO_VAL_LABEL, txt_file)
            
            with open(txt_path, 'w') as f:
                for annotation in yolo_annotations:
                    f.write(' '.join(map(str, annotation)) + '\n')
        
        # Create dataset configuration file
        self.create_dataset_yaml()
        
        print(f"YOLO dataset created successfully!")
        
    def create_dataset_yaml(self):
        """Create YOLO dataset configuration file"""
        dataset_config = {
            'path': os.path.abspath(YOLO_DATASET_ROOT),
            'train': 'images/train',
            'val': 'images/val',
            'names': {i: name for name, i in LABEL_MAP.items()},
            'nc': NUM_CLASSES
        }
        
        yaml_path = os.path.join(YOLO_DATASET_ROOT, 'dataset.yaml')
        with open(yaml_path, 'w') as f:
            yaml.dump(dataset_config, f, default_flow_style=False)
        
        return yaml_path

class ImprovedYOLOTrainer:
    """Enhanced YOLO trainer with advanced features"""
    
    def __init__(self, model_name='yolov8n.pt', img_size=640):
        self.model_name = model_name
        self.img_size = img_size
        self.model = None
        
    def load_model(self):
        """Load YOLO model with custom modifications"""
        self.model = YOLO(self.model_name)
        
        # Modify model architecture for better performance
        # Add dropout layers and modify head architecture
        if hasattr(self.model.model, 'model'):
            # Add custom modifications here
            print("Model loaded and ready for training")
        
    def create_augmentation_pipeline(self):
        """Create advanced data augmentation pipeline"""
        transform = A.Compose([
            A.RandomRotate90(p=0.3),
            A.Flip(p=0.3),
            A.RandomBrightnessContrast(
                brightness_limit=0.2, 
                contrast_limit=0.2, 
                p=0.5
            ),
            A.HueSaturationValue(
                hue_shift_limit=10,
                sat_shift_limit=15,
                val_shift_limit=10,
                p=0.3
            ),
            A.GaussNoise(var_limit=10, p=0.2),
            A.MotionBlur(blur_limit=3, p=0.2),
            A.CLAHE(clip_limit=2.0, p=0.3),
            A.ShiftScaleRotate(
                shift_limit=0.1,
                scale_limit=0.1,
                rotate_limit=15,
                border_mode=cv2.BORDER_CONSTANT,
                value=0,
                p=0.3
            ),
        ], bbox_params=BboxParams(format='yolo', label_fields=['class_labels']))
        
        return transform
    
    def train_model(self, data_yaml, epochs=100, batch_size=16, patience=20):
        """Train YOLO model with advanced configurations"""
        if self.model is None:
            self.load_model()
        
        # Training hyperparameters
        training_args = {
            'data': data_yaml,
            'epochs': epochs,
            'batch': batch_size,
            'imgsz': self.img_size,
            'device': DEVICE,
            'patience': patience,
            'save_period': 10,
            'workers': 4,
            'project': 'runs/detect',
            'name': 'neudet_improved',
            'exist_ok': True,
            
            # Optimization settings
            'optimizer': 'AdamW',
            'lr0': 0.001,
            'lrf': 0.01,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            
            # Augmentation settings
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 15.0,
            'translate': 0.1,
            'scale': 0.5,
            'shear': 0.0,
            'perspective': 0.0,
            'flipud': 0.0,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.1,
            'copy_paste': 0.1,
            
            # Loss settings
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            
            # Other settings
            'val': True,
            'plots': True,
            'save_json': True,
            'verbose': True
        }
        
        print("Starting training with improved YOLO model...")
        results = self.model.train(**training_args)
        
        # Save the best model
        best_model_path = f"runs/detect/neudet_improved/weights/best.pt"
        print(f"Training completed! Best model saved at: {best_model_path}")
        
        return results, best_model_path

class InferenceEngine:
    """Enhanced inference engine for predictions"""
    
    def __init__(self, model_path, conf_threshold=0.25, iou_threshold=0.45):
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        
    def predict_and_export(self, test_dir, save_csv_path, save_img_dir="output", 
                          visualize=True, tta=False):
        """Enhanced prediction with TTA and better post-processing"""
        os.makedirs(save_img_dir, exist_ok=True)
        rows = []
        
        test_images = sorted([f for f in os.listdir(test_dir) 
                            if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        
        print(f"Processing {len(test_images)} test images...")
        
        for img_file in tqdm(test_images, desc="Predicting"):
            img_path = os.path.join(test_dir, img_file)
            
            # Run inference with optional TTA
            if tta:
                results = self.predict_with_tta(img_path)
            else:
                results = self.model(img_path, 
                                   conf=self.conf_threshold,
                                   iou=self.iou_threshold,
                                   imgsz=IMG_SIZE,
                                   device=DEVICE,
                                   augment=True)
            
            # Process results
            if len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes
                
                if len(boxes) > 0:
                    # Extract predictions
                    conf_scores = boxes.conf.cpu().numpy()
                    class_ids = boxes.cls.cpu().numpy().astype(int)
                    bbox_coords = boxes.xyxy.cpu().numpy()
                    
                    # Apply additional filtering
                    valid_indices = self.apply_post_processing(
                        bbox_coords, conf_scores, class_ids
                    )
                    
                    if len(valid_indices) > 0:
                        conf_scores = conf_scores[valid_indices]
                        class_ids = class_ids[valid_indices]
                        bbox_coords = bbox_coords[valid_indices]
                        
                        # Create submission row
                        row = self.create_submission_row(
                            img_file, bbox_coords, conf_scores, class_ids
                        )
                        rows.append(row)
                        
                        # Visualize if requested
                        if visualize:
                            self.visualize_predictions(
                                img_path, bbox_coords, conf_scores, class_ids, 
                                save_img_dir, img_file
                            )
                    else:
                        # No valid detections
                        rows.append(self.create_empty_row(img_file))
                else:
                    # No detections
                    rows.append(self.create_empty_row(img_file))
            else:
                # No detections
                rows.append(self.create_empty_row(img_file))
        
        # Save results to CSV
        df = pd.DataFrame(rows)
        df.fillna("none", inplace=True)
        df.to_csv(save_csv_path, index=False)
        
        print(f"[INFO] Saved CSV to {save_csv_path}")
        if visualize:
            print(f"[INFO] Saved annotated images to {save_img_dir}/")
        
        # Print detection statistics
        self.print_detection_stats(df)
        
    def predict_with_tta(self, img_path):
        """Test Time Augmentation for better predictions"""
        transforms = [
            lambda x: x,  # Original
            lambda x: cv2.flip(x, 1),  # Horizontal flip
            lambda x: cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE),  # 90° rotation
            lambda x: cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE),  # -90° rotation
        ]
        
        img = cv2.imread(img_path)
        all_results = []
        
        for transform in transforms:
            transformed_img = transform(img.copy())
            temp_path = "temp_tta.jpg"
            cv2.imwrite(temp_path, transformed_img)
            
            results = self.model(temp_path,
                               conf=self.conf_threshold,
                               iou=self.iou_threshold,
                               imgsz=IMG_SIZE,
                               device=DEVICE)
            all_results.extend(results)
        
        # Clean up temp file
        if os.path.exists("temp_tta.jpg"):
            os.remove("temp_tta.jpg")
        
        # Ensemble results (simplified - in practice, you'd want proper coordinate transformation)
        return all_results[:1]  # Return first result for now
        
    def apply_post_processing(self, boxes, scores, classes):
        """Apply additional post-processing filters"""
        valid_indices = []
        
        for i, (box, score, cls) in enumerate(zip(boxes, scores, classes)):
            x1, y1, x2, y2 = box
            width = x2 - x1
            height = y2 - y1
            area = width * height
            
            # Filter by minimum area and aspect ratio
            if area > 100 and 0.1 < width/height < 10:
                valid_indices.append(i)
        
        return valid_indices
    
    def create_submission_row(self, img_file, boxes, scores, classes):
        """Create submission row with proper formatting"""
        # Take the highest confidence detection as primary label
        best_idx = np.argmax(scores)
        primary_label = REV_LABEL_MAP[classes[best_idx]]
        
        # Format all detections
        cf_list = [f"{score:.3f}" for score in scores]
        xmin_list = [str(int(box[0])) for box in boxes]
        ymin_list = [str(int(box[1])) for box in boxes]
        xmax_list = [str(int(box[2])) for box in boxes]
        ymax_list = [str(int(box[3])) for box in boxes]
        
        return {
            "ID": img_file,
            "label": primary_label,
            "cf": " ".join(cf_list),
            "xmin": " ".join(xmin_list),
            "ymin": " ".join(ymin_list),
            "xmax": " ".join(xmax_list),
            "ymax": " ".join(ymax_list),
        }
    
    def create_empty_row(self, img_file):
        """Create empty row for images with no detections"""
        return {
            "ID": img_file,
            "label": "none",
            "cf": "0",
            "xmin": "0",
            "ymin": "0",
            "xmax": "0",
            "ymax": "0",
        }
    
    def visualize_predictions(self, img_path, boxes, scores, classes, 
                            save_dir, img_file):
        """Visualize predictions with improved styling"""
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(image)
        
        colors = plt.cm.Set3(np.linspace(0, 1, NUM_CLASSES))
        
        for box, score, cls in zip(boxes, scores, classes):
            x1, y1, x2, y2 = box
            width = x2 - x1
            height = y2 - y1
            
            color = colors[cls]
            
            # Draw bounding box
            rect = patches.Rectangle((x1, y1), width, height,
                                   linewidth=2, edgecolor=color, 
                                   facecolor='none', alpha=0.8)
            ax.add_patch(rect)
            
            # Add label with background
            label_text = f"{REV_LABEL_MAP[cls]}: {score:.3f}"
            ax.text(x1, y1 - 5, label_text,
                   bbox=dict(boxstyle="round,pad=0.3", facecolor=color, alpha=0.7),
                   fontsize=10, color='black', weight='bold')
        
        ax.set_title(f"Detections: {img_file}", fontsize=14, weight='bold')
        ax.axis('off')
        
        # Save with high quality
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, img_file), 
                   dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()
    
    def print_detection_stats(self, df):
        """Print detection statistics"""
        total_images = len(df)
        images_with_detections = len(df[df['label'] != 'none'])
        
        print(f"\n=== Detection Statistics ===")
        print(f"Total images processed: {total_images}")
        print(f"Images with detections: {images_with_detections}")
        print(f"Images without detections: {total_images - images_with_detections}")
        print(f"Detection rate: {images_with_detections/total_images*100:.1f}%")
        
        # Class distribution
        if images_with_detections > 0:
            class_counts = df[df['label'] != 'none']['label'].value_counts()
            print(f"\nClass distribution:")
            for class_name, count in class_counts.items():
                print(f"  {class_name}: {count}")

def main():
    """Main execution function"""
    print("=== NEU-DET Object Detection with Improved YOLO ===")
    
    # Step 1: Convert dataset to YOLO format
    print("\n1. Converting dataset to YOLO format...")
    converter = DatasetConverter(TRAIN_IMG_ROOT, TRAIN_ANN_ROOT)
    converter.create_yolo_dataset(train_split=0.85)
    
    # Step 2: Initialize and train model
    print("\n2. Initializing YOLO trainer...")
    trainer = ImprovedYOLOTrainer(model_name='yolov8m.pt', img_size=IMG_SIZE)
    
    dataset_yaml = os.path.join(YOLO_DATASET_ROOT, 'dataset.yaml')
    
    print("\n3. Starting training...")
    results, best_model_path = trainer.train_model(
        data_yaml=dataset_yaml,
        epochs=150,
        batch_size=16,
        patience=25
    )
    
    # Step 3: Run inference
    print("\n4. Running inference on test set...")
    inference_engine = InferenceEngine(
        model_path=best_model_path,
        conf_threshold=CONF_THRESHOLD,
        iou_threshold=IOU_THRESHOLD
    )
    
    inference_engine.predict_and_export(
        test_dir=VAL_IMG_ROOT,
        save_csv_path=SUBMISSION_PATH,
        save_img_dir="output_visualizations",
        visualize=True,
        tta=False  # Set to True for Test Time Augmentation
    )
    
    print("\n=== Training and Inference Complete! ===")
    print(f"Submission file saved to: {SUBMISSION_PATH}")
    print("Check the runs/detect/neudet_improved/ folder for training logs and metrics.")

if __name__ == "__main__":
    # Install required packages if not already installed
    try:
        import ultralytics
        import albumentations
    except ImportError:
        print("Installing required packages...")
        os.system("pip install ultralytics albumentations")
        print("Packages installed. Please restart and run again.")
        exit()
    
    main()

/raid/jupyter-lcstw596113/.local/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Using device: cuda
=== NEU-DET Object Detection with Improved YOLO ===

1. Converting dataset to YOLO format...
Converting dataset: 979 train, 173 val


Converting val set: 100%|██████████| 173/173 [00:00<00:00, 601.11it/s]


YOLO dataset created successfully!

2. Initializing YOLO trainer...

3. Starting training...
Model loaded and ready for training
Starting training with improved YOLO model...
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./yolo_dataset/dataset.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=neudet_improved, nbs=64, nms=False, opset=None, optimize=Fal

train: Scanning /mnt/jp/LCS/Machine Learning/Final Project/yolo_dataset/labels/train... 1363 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1363/1363 [00:00<00:00, 2820.43it/s]

...


image 1/1 /mnt/jp/LCS/Machine Learning/Final Project/test/scratches_ymyqf6larhda.jpg: 640x640 3 scratchess, 46.3ms
Speed: 3.9ms preprocess, 46.3ms inference, 15.0ms postprocess per image at shape (1, 3, 640, 640)


Predicting: 100%|█████████▉| 359/360 [01:12<00:00,  5.02it/s]


image 1/1 /mnt/jp/LCS/Machine Learning/Final Project/test/scratches_z81i6li74hhg.jpg: 640x640 1 inclusion, 1 scratches, 46.3ms
Speed: 2.1ms preprocess, 46.3ms inference, 11.1ms postprocess per image at shape (1, 3, 640, 640)


Predicting: 100%|██████████| 360/360 [01:13<00:00,  4.92it/s]


[INFO] Saved CSV to ./submission_Yolo_1.csv
[INFO] Saved annotated images to output_visualizations/

=== Detection Statistics ===
Total images processed: 360
Images with detections: 358
Images without detections: 2
Detection rate: 99.4%

Class distribution:
  patches: 69
  inclusion: 63
  crazing: 60
  rolled-in_scale: 60
  scratches: 56
  pitted_surface: 50

=== Training and Inference Complete! ===
Submission file saved to: ./submission_Yolo_1.csv
Check the runs/detect/neudet_improved/ folder for training logs and metrics.
